In [1]:
#imports
!pip install wikipedia
!pip install fasttext
!pip install gensim
!pip install nltk

import string
import pandas as pd
import wikipedia
import fasttext
import fasttext.util
from pprint import pprint as print
from gensim.models.fasttext import FastText as fasttext
from gensim.test.utils import datapath
from google.colab import drive
import xml.etree.ElementTree as ET
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
import fasttext.util
import sklearn as sk
from sklearn import svm
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=5e2608a6cada8a024ec98b77af002d1d82f12cdb15699f041c4db317222de7ff
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 71kB 1.6MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3019109 sha256=89f4e80d1582f2365dbe90c143fa598b57da65cdeef8b3d51905eb8cd2691108
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [2]:
#initial setup
nltk.download('punkt')
nltk.download('stopwords')

fasttext.util.download_model('it', if_exists='ignore') 
ft = fasttext.load_model('cc.it.300.bin')

wikipedia.set_lang("it")

drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [3]:
#dataset init

#file consisting of pairs of target and prerequisite concepts (A, B) labelled as follows:
#1 if B is a prerequisite of A;
#0 in all other cases.
df = pd.read_csv('/content/drive/My Drive/train.csv')
train = df.sample(frac=0.8,random_state=200) 
validation = df.drop(train.index)

#The Wikipedia page of each concept found in the previous file.
#Each Wikipedia page is introduced by a `<doc>` element (with *id* and *url*) 
#containing the title and the text of the corresponding page.
tree = ET.parse('/content/drive/My Drive/dataset.xml')
pages = tree.getroot()

In [5]:
#create dictionary of tokenized documents
punct = string.punctuation + '«``»' + "''"
it_stop_words = nltk.corpus.stopwords.words('italian')
stemmer = nltk.stem.snowball.ItalianStemmer(True)

doc_dict = {}

for content in pages.iter('doc'):
  #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
  document = (content.find('title').text  + content.find('text').text).replace("'"," ")
  #tokenized
  tokenized = nltk.tokenize.word_tokenize(document, "italian")
  #no punctuation and lowercase
  no_punct = [x.lower() for x in tokenized if x not in punct]
  #remove stop words
  no_stop_words = [x for x in no_punct if x not in it_stop_words]
  #stem
  #stemmed = [stemmer.stem(i) for i in no_stop_words]
  
  #add tokenized document
  doc_dict[content.find('title').text] = no_stop_words

In [23]:
#create doc embeddings with fasttext by averaging word embeddings
weight_A = 1
weight_B = 1

#create train dataset

ft_train = {k: [] for k in range(600)}
ft_train['prerequisite'] = []
for index, row in train.iterrows():
    A = doc_dict[row[0]]
    B = doc_dict[row[1]]
    doc_embedding_A = np.zeros(300)
    doc_embedding_B = np.zeros(300)

    for word in A:
      word_embedding = np.array(ft.get_word_vector(word))
      doc_embedding_A = doc_embedding_A + word_embedding
    for word in B:
      word_embedding = np.array(ft.get_word_vector(word))
      doc_embedding_B = doc_embedding_B + word_embedding  
    
    #averaged doc_embeddings
    doc_embedding_A = doc_embedding_A / len(A)
    doc_embedding_B = doc_embedding_B / len(B)

    data = np.concatenate([weight_A * doc_embedding_A, weight_B * doc_embedding_B]) 
    for i,val in enumerate(data):
      ft_train[i].append(val)
    ft_train['prerequisite'].append(row[2])

#create validation dataset

ft_validation = {k: [] for k in range(600)}
ft_validation['prerequisite'] = []
for index, row in validation.iterrows():
    A = doc_dict[row[0]]
    B = doc_dict[row[1]]
    doc_embedding_A = np.zeros(300)
    doc_embedding_B = np.zeros(300)

    for word in A:
      word_embedding = np.array(ft.get_word_vector(word))
      doc_embedding_A = doc_embedding_A + word_embedding
    for word in B:
      word_embedding = np.array(ft.get_word_vector(word))
      doc_embedding_B = doc_embedding_B + word_embedding  
    
    #averaged doc_embeddings
    doc_embedding_A = doc_embedding_A / len(A)
    doc_embedding_B = doc_embedding_B / len(B)

    data = np.concatenate([weight_A * doc_embedding_A, weight_B * doc_embedding_B]) 
    for i,val in enumerate(data):
      ft_validation[i].append(val)
    ft_validation['prerequisite'].append(row[2])

ft_df_train = pd.DataFrame(data = ft_train)
ft_df_validation = pd.DataFrame(data = ft_validation)

y_train = ft_df_train.iloc[:,600]
X_train = ft_df_train.iloc[:,:600]

X_test = ft_df_validation.iloc[:,:600]
y_test = ft_df_validation.iloc[:,600]

In [ ]:
#create doc embeddings with fasttext by averaging word embeddings
#but with a higher weight to the words that make up the title in B

#intuition:
#if I find in A the words that make up the title in B, then there's
#a good chance that B is a prerequisite of A

#examples of intuition:
#the wikipedia page of "Light" is a prerequisite of the wikipedia page of "Total internal reflection"
#the wikipedia page talking about "Total internal reflection" mentions the word "light" many times
#"Magnet" is a prerequisite of "Magnetic field" 
#the word "magnet" appears various times in the "Magnetic field" page

#the weight given to the word depends on the length of the document A
#the bigger the document the bigger the weight
#this is to prevent small documents from being too dependant on the word
#and for the word to be irrelevant in big documents

word_percent = 0.1

#create train dataset

ft_train = {k: [] for k in range(600)}
ft_train['prerequisite'] = []
for index, row in train.iterrows():
    A = doc_dict[row[0]]
    B_title = row[1]
    B = doc_dict[B_title]
    doc_embedding_A = np.zeros(300)
    doc_embedding_B = np.zeros(300)

    #clean the title
    B_title = B_title.replace("'"," ")
    #tokenized
    B_title_tokenized = nltk.tokenize.word_tokenize(B_title, "italian")
    #no punctuation and lowercase
    B_title_tokenized_no_punct = [x.lower() for x in B_title_tokenized if x not in punct]
    #remove stop words
    B_title = [x for x in B_title_tokenized_no_punct if x not in it_stop_words]

    for word in A:
      weight = 1
      if word in B_title:
        weight = 1
        weight = len(A) * word_percent
      word_embedding = weight * np.array(ft.get_word_vector(word))
      doc_embedding_A = doc_embedding_A + word_embedding

    for word in B:
      weight = 1
      if word in B_title:
        weight = 1
        #weight = len(B) * word_percent
      word_embedding = weight * np.array(ft.get_word_vector(word))
      doc_embedding_B = doc_embedding_B + word_embedding  
    
    #averaged doc_embeddings
    doc_embedding_A = doc_embedding_A / len(A)
    doc_embedding_B = doc_embedding_B / len(B)

    data = np.concatenate([doc_embedding_A,doc_embedding_B]) 
    for i,val in enumerate(data):
      ft_train[i].append(val)
    ft_train['prerequisite'].append(row[2])

#create validation dataset

ft_validation = {k: [] for k in range(600)}
ft_validation['prerequisite'] = []
for index, row in validation.iterrows():
    A = doc_dict[row[0]]
    B_title = row[1]
    B = doc_dict[B_title]
    doc_embedding_A = np.zeros(300)
    doc_embedding_B = np.zeros(300)

    #clean the title
    B_title = B_title.replace("'"," ")
    #tokenized
    B_title_tokenized = nltk.tokenize.word_tokenize(B_title, "italian")
    #no punctuation and lowercase
    B_title_tokenized_no_punct = [x.lower() for x in B_title_tokenized if x not in punct]
    #remove stop words
    B_title = [x for x in B_title_tokenized_no_punct if x not in it_stop_words]

    for word in A:
      weight = 1
      if word in B_title:
        weight = 1
        weight = len(A) * word_percent
      word_embedding = weight * np.array(ft.get_word_vector(word))
      doc_embedding_A = doc_embedding_A + word_embedding

    for word in B:
      weight = 1
      if word in B_title:
        weight = 1
        #weight = len(B) * word_percent
      word_embedding = weight * np.array(ft.get_word_vector(word))
      doc_embedding_B = doc_embedding_B + word_embedding  
    
    #averaged doc_embeddings
    doc_embedding_A = doc_embedding_A / len(A)
    doc_embedding_B = doc_embedding_B / len(B)

    data = np.concatenate([doc_embedding_A,doc_embedding_B]) 
    for i,val in enumerate(data):
      ft_validation[i].append(val)
    ft_validation['prerequisite'].append(row[2])

ft_df_train = pd.DataFrame(data = ft_train)
ft_df_validation = pd.DataFrame(data = ft_validation)

y_train = ft_df_train.iloc[:,600]
X_train = ft_df_train.iloc[:,:600]

X_test = ft_df_validation.iloc[:,:600]
y_test = ft_df_validation.iloc[:,600]

In [24]:
#Logistic regression

#train
LR = LogisticRegression(max_iter=1000)
LR.fit(X_train, y_train)

#validate
y_pred = LR.predict(X_test)

print('Accuracy score: {:3f}'.format(accuracy_score(y_test, y_pred)))
print('Precision score: {:3f}'.format(precision_score(y_test, y_pred)))
print('Recall score: {:3f}'.format(recall_score(y_test, y_pred)))
print('F1 score: {:3f}'.format(f1_score(y_test, y_pred)))

'Accuracy score: 0.824725'
'Precision score: 0.857143'
'Recall score: 0.028302'
'F1 score: 0.054795'


In [25]:
#Linear SVM

#train
SVM = svm.LinearSVC(max_iter=10000,C=500)
SVM.fit(X_train, y_train)

#validate
y_pred = SVM.predict(X_test)

print('Accuracy score: {:3f}'.format(accuracy_score(y_test, y_pred)))
print('Precision score: {:3f}'.format(precision_score(y_test, y_pred)))
print('Recall score: {:3f}'.format(recall_score(y_test, y_pred)))
print('F1 score: {:3f}'.format(f1_score(y_test, y_pred)))


'Accuracy score: 0.920406'
'Precision score: 0.801020'
'Recall score: 0.740566'
'F1 score: 0.769608'


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [26]:
#rbf SVM

#train
SVM = svm.SVC(max_iter=10000,C=500, gamma=10)
SVM.fit(X_train, y_train)

#validate
y_pred = SVM.predict(X_test)

print('Accuracy score: {:3f}'.format(accuracy_score(y_test, y_pred)))
print('Precision score: {:3f}'.format(precision_score(y_test, y_pred)))
print('Recall score: {:3f}'.format(recall_score(y_test, y_pred)))
print('F1 score: {:3f}'.format(f1_score(y_test, y_pred)))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


'Accuracy score: 0.933108'
'Precision score: 0.818182'
'Recall score: 0.806604'
'F1 score: 0.812352'


In [27]:
#Random Forest

#train
RF = RandomForestClassifier(max_features = None)
RF.fit(X_train, y_train)

#validate
y_pred = RF.predict(X_test)

print('Accuracy score: {:3f}'.format(accuracy_score(y_test, y_pred)))
print('Precision score: {:3f}'.format(precision_score(y_test, y_pred)))
print('Recall score: {:3f}'.format(recall_score(y_test, y_pred)))
print('F1 score: {:3f}'.format(f1_score(y_test, y_pred)))

'Accuracy score: 0.932261'
'Precision score: 0.892857'
'Recall score: 0.707547'
'F1 score: 0.789474'
